# Transacciones activas ~ Adam Machanic/SP_WhoIsActive

Muestra las tareas que actualmente se encuentran en ejecución.

> Mas información en → http://whoisactive.com/docs/

In [ ]:
exec sp_WhoIsActive 
     @get_transaction_info = 1            -- Enables pulling transaction log write info and transaction duration
    ,@get_outer_command = 1               -- Get the associated outer ad hoc query or stored procedure call, if available
    ,@get_plans = 1                       -- Get associated query plans for running tasks, if available

# Evaluación de desempeño en tiempo real ~ Brent Ozar/SP_BlitzFirst

Toma dos muestras con información del estado del servidor y las compara, buscando diagnosticar cualquier problema de desempeño que se pueda estar presentando

> Más información en → https://bit.ly/3arz8lo

In [ ]:
exec sp_blitzfirst 
	 @Seconds = 10                        -- Change if you need more time space for analysis
    ,@CheckServerInfo = 1                 -- Disable if you don't need additional server information
    ,@ExpertMode = 1                      -- For extra diagnistic information
    -- ,@ShowSleepingSPIDs = 1            -- Shows long-running sleeping queries that might be blocking others
    -- ,@CheckProcedureCache = 1          -- Analyzes execution plans based on query cache
go

# Analizar cache de planes de ejecución ~ Brent Ozar/SP_BlitzCache


Organiza las consultas recientemente ejecutadas basándose en el impacto que han tenido en la instancia

> Más información en → https://bit.ly/2WWV9EL

In [ ]:
exec sp_blitzcache
     @Top = 10                              -- Show TOP querys
    -- ,@ExpertMode = 1                     -- Enable expert mode for advanced information
    -- ,@IgnoreSystemDBs = 1                -- Analyze system databases
    -- ,@DatabaseName = 'test'              -- Analyze query cache based on database execution context

# Desempeño de archivos de TempDB

Muestra el desempeño de los archivos de datos de la TempDB

> Para información adicional revisar los datos en /performance/ss-db_file_performance

In [ ]:
select files.physical_name
      ,files.name
      ,stats.num_of_writes
      ,(1.0 * stats.io_stall_write_ms / stats.num_of_writes) as avg_write_stall_ms
      ,stats.num_of_reads
      ,(1.0 * stats.io_stall_read_ms / stats.num_of_reads) as avg_read_stall_ms
  from sys.dm_io_virtual_file_stats(2, null) as stats
  join master.sys.master_files as files
    on stats.database_id = files.database_id
   and stats.file_id = files.file_id
 where files.type_desc = 'ROWS'

# Desempeño de archivos de datos y log

Muestra el desempeño de los archivos de datos para cada base de datos de la instancia

In [ ]:
declare @sysdb int = 0; -- If 1 retrieve system databases information
select m.name as database_name
      ,files.physical_name
	  ,files.type_desc
      ,files.name file_name
      ,stats.num_of_writes
      ,case when stats.io_stall_write_ms <> 0 then (1.0 * stats.io_stall_write_ms / stats.num_of_writes) end as avg_write_stall_ms
      ,stats.num_of_reads
      ,case when stats.io_stall_read_ms <> 0 then (1.0 * stats.io_stall_read_ms / stats.num_of_reads) end as avg_read_stall_ms
  from sys.dm_io_virtual_file_stats(NULL, NULL) as stats
  join master.sys.master_files as files
    on stats.database_id = files.database_id
   and stats.file_id = files.file_id
  join master.sys.databases m
    on m.database_id = files.database_id
 where 1=1
    -- and files.type_desc = 'ROWS'
   and m.database_id > case when @sysdb = 0 then 4 else 0 end
 order by 1 desc;

# Bloqueos en la instancia ~ Brent Ozar/SP_BlitzLock

Comprueba los LOGs del sistema que capturan Dead Locks, analizando todo el XML para retornar resultados legibles

> Mas información en → https://bit.ly/2K3PAwX

In [ ]:
exec sp_BlitzLock
	 @Top = 10                            -- Use if you want to limit the number of deadlocks to return. This is ordered by event date ascending
    -- ,@DatabaseName:                    -- If you want to filter to a specific database
    -- ,@StartDate:                       -- The date you want to start searching on
    -- ,@EndDate:                         -- The date you want to stop searching on
    -- ,@ObjectName:                      -- If you want to filter to a specific table. The object name has to be fully qualified 'Database.Schema.Table'
    -- ,@StoredProcName:                  -- If you want to search for a single stored procedure. Don't specify a schema or database name - just a stored procedure name alone is all you need, and if it exists in any schema (or multiple schemas), we'll find it
    -- ,@AppName:                         -- If you want to filter to a specific application
    -- ,@HostName:                        -- If you want to filter to a specific host
    -- ,@LoginName:                       -- If you want to filter to a specific login
    -- ,@EventSessionPath:                -- If you want to point this at an XE session rather than the system health session
go